In [1]:
!pip install crewai==0.22.5
!pip install exa_py==1.0.9
!pip install langchain==0.1.13

In [2]:
import os
from exa_py import Exa
from langchain.agents import tool
from textwrap import dedent
from IPython.display import Markdown
from crewai import Task, Crew, Agent
from langchain_openai import ChatOpenAI

In [3]:
os.environ["OPENAI_API_KEY"] = "" # Input your openai api key

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.5
)

In [4]:
class ExaSearchToolset():
  @tool
  def search(query: str):
    """Search for a webpage based on the query."""
    return ExaSearchToolset._exa().search(f"{query}", use_autoprompt=True, num_results=3)

  @tool
  def find_similar(url: str):
      """Search for webpages similar to a given URL.
      The url passed in should be a URL returned from `search`.
      """
      return ExaSearchToolset._exa().find_similar(url, num_results=3)

  @tool
  def get_contents(ids: str):
      """Get the contents of a webpage.
      The ids must be passed in as a list, a list of ids returned from `search`.
      """
      ids = eval(ids)
      contents = str(ExaSearchToolset._exa().get_contents(ids))
      contents = contents.split("URL:")
      contents = [content[:1000] for content in contents]
      return "\n\n".join(contents)

  def tools():
    return [
      ExaSearchToolset.search,
      ExaSearchToolset.find_similar,
      ExaSearchToolset.get_contents
    ]


  def _exa():
    return Exa(api_key="")  # get your Exa api key from https://docs.exa.ai/reference/getting-an-api-key and https://dashboard.exa.ai/api-keys

In [5]:
class MeetingPrepTasks():
  def research_task(self, agent, meeting_participants, meeting_context):
    return Task(
        description=dedent(f"""\
        Conduct comprehensive research on each of the individuals and companies
                involved in the upcoming meeting. Gather information on recent
                news, achievements, professional background, and any relevant
                business activities.

                Participants: {meeting_participants}
                Meeting Context: {meeting_context}"""),
        expected_output=dedent(f"""\
        A detailed report summarizing key findings about each participant
                and company, highlighting information that could be relevant for the meeting."""),
        agent=agent,
        async_execution=True
    )

  def industry_analysis_task(self, agent, meeting_participants, meeting_context):
    return Task(
        description=dedent(f"""\
            Analyze the current industry trends, challenges, and opportunities
            relevant to the meeting's context. Consider market reports, recent
            developments, and expert opinions to provide a comprehensive
            overview of the industry landscape.

            Participants: {meeting_participants}
            Meeting Context: {meeting_context}"""),
        expected_output=dedent("""\
            An insightful analysis that identifies major trends, potential
            challenges, and strategic opportunities."""),
        async_execution=True,
        agent=agent
    )

  def meeting_strategy_task(self, agent, meeting_context, meeting_objective):
    return Task(
        description=dedent(f"""\
            Develop strategic talking points, questions, and discussion angles
            for the meeting based on the research and industry analysis conducted

            Meeting Context: {meeting_context}
            Meeting Objective: {meeting_objective}"""),
        expected_output=dedent("""\
            Complete report with a list of key talking points, strategic questions
            to ask to help achieve the meetings objective during the meeting."""),
        agent=agent
    )

  def summary_and_briefing_task(self, agent, meeting_context, meeting_objective):
    return Task(
        description=dedent(f"""\
            Compile all the research findings, industry analysis, and strategic
            talking points into a concise, comprehensive briefing document for
            the meeting.
            Ensure the briefing is easy to digest and equips the meeting
            participants with all necessary information and strategies.

            Meeting Context: {meeting_context}
            Meeting Objective: {meeting_objective}"""),
        expected_output=dedent("""\
            A well-structured briefing document that includes sections for
            participant bios, industry overview, talking points, and
            strategic recommendations."""),
        agent=agent
    )

In [6]:
class MeetingPrepAgents():
    def research_agent(self):
      return Agent(
        role="Research Specialist",
        goal='Conduct thorough research on people and companies involved in the meeting',
        tools=ExaSearchToolset.tools(),
        backstory=dedent("""\
                As a Research Specialist, your mission is to uncover detailed information
                about the individuals and entities participating in the meeting. Your insights
                will lay the groundwork for strategic meeting preparation."""),
        llm=llm,
        verbose=True
      )

    def industry_analysis_agent(self):
      return Agent(
        role='Industry Analyst',
        goal='Analyze the current industry trends, challenges, and opportunities',
        tools=ExaSearchToolset.tools(),
        backstory=dedent("""\
            As an Industry Analyst, your analysis will identify key trends,
            challenges facing the industry, and potential opportunities that
            could be leveraged during the meeting for strategic advantage."""),
        llm=llm,
        verbose=True
      )

    def meeting_strategy_agent(self):
      return Agent(
        role='Meeting Strategy Advisor',
        goal='Develop talking points, questions, and strategic angles for the meeting',
        backstory=dedent("""\
            As a Strategy Advisor, your expertise will guide the development of
            talking points, insightful questions, and strategic angles
            to ensure the meeting's objectives are achieved."""),
        llm=llm,
        verbose=True
      )

    def summary_and_briefing_agent(self):
      return Agent(
        role='Briefing Coordinator',
        goal='Compile all gathered information into a concise, informative briefing document',
        backstory=dedent("""\
            As the Briefing Coordinator, your role is to consolidate the research,
            analysis, and strategic insights."""),
        llm=llm,
        verbose=True
      )

In [7]:
print("## Welcome to the Meeting Prep Crew")
print('-------------------------------')
meeting_participants = input("What are the emails for the participants (other than you) in the meeting?\n")
meeting_context = input("What is the context of the meeting?\n")
meeting_objective = input("What is your objective for this meeting?\n")

tasks = MeetingPrepTasks()
agents = MeetingPrepAgents()

# create agents
research_agent = agents.research_agent()
industry_analysis_agent = agents.industry_analysis_agent()
meeting_strategy_agent = agents.meeting_strategy_agent()
summary_and_briefing_agent = agents.summary_and_briefing_agent()

# create tasks
research_task = tasks.research_task(research_agent, meeting_participants, meeting_context)
industry_analysis_task = tasks.industry_analysis_task(industry_analysis_agent, meeting_participants, meeting_context)
meeting_strategy_task = tasks.meeting_strategy_task(meeting_strategy_agent, meeting_context, meeting_objective)
summary_and_briefing_task = tasks.summary_and_briefing_task(summary_and_briefing_agent, meeting_context, meeting_objective)

meeting_strategy_task.context = [research_task, industry_analysis_task]
summary_and_briefing_task.context = [research_task, industry_analysis_task, meeting_strategy_task]

crew = Crew(
    agents=[
    research_agent,
    industry_analysis_agent,
    meeting_strategy_agent,
    summary_and_briefing_agent
    ],
    tasks=[
    research_task,
    industry_analysis_task,
    meeting_strategy_task,
    summary_and_briefing_task
    ]
)

result = crew.kickoff()

print(result)

## Welcome to the Meeting Prep Crew
-------------------------------
What are the emails for the participants (other than you) in the meeting?
li@gmail.com
What is the context of the meeting?
large language model
What is your objective for this meeting?
the impact of large language model to human life


> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
I need to start by gathering information on li@gmail.com and any relevant companies they are associated with. I will need to search for recent news, achievements, professional backgrounds, and business activities.

Action: search
Action Input: {"query": "li@gmail.com"}I need to gather information on the current industry trends, challenges, and opportunities relevant to the meeting's context of a large language model. I should search for market reports, recent developments, and expert opinions to provide a comprehensive overview.

Action: search
Action Input: {"query": "current industry trends in large l

In [9]:
Markdown(result)

**Briefing Document: Impact of Large Language Models on Human Life**

**Participant Bios:**
- [Research Specialist]
- [Industry Analyst]
- [Meeting Strategy Advisor]

**Industry Overview:**
Large language models (LLMs) have emerged as a groundbreaking technology in artificial intelligence, capable of generating human-like text based on input data. The rise of generative AI LLMs such as ChatGPT has revolutionized the way we interact with language and information. Companies are investing heavily in developing and improving LLMs to enhance language understanding and generation capabilities. However, ethical concerns around data privacy, bias in language generation, and scalability challenges remain key considerations in the industry. Despite these challenges, LLMs offer opportunities for personalized content generation, language translation and understanding, and global communication advancements.

**Talking Points:**
1. The Rise of Generative AI Large Language Models: Discuss the capabilities and popularity of LLMs like ChatGPT in generating coherent and contextually relevant text.
2. AI's Next Frontier: Investment in Large Language Models: Highlight the importance of investing in LLMs to enhance language understanding and generation.
3. Ethical Concerns: Address the ethical considerations regarding data privacy, bias, and potential misuse of AI-generated content.
4. Scalability Challenges: Explore the challenges in building and maintaining LLMs due to computational resources and expertise requirements.
5. Opportunities for Personalized Content Generation: Emphasize the benefits of using LLMs to create personalized content for improved user engagement.
6. Language Translation and Understanding: Discuss the potential improvements in language translation and understanding through advancements in LLMs.

**Strategic Recommendations:**
1. Implement robust data privacy measures to address ethical concerns related to LLMs.
2. Invest in computational resources and expertise to overcome scalability challenges in building and maintaining LLMs.
3. Develop partnerships and research initiatives to further advance LLM capabilities and applications.
4. Conduct thorough impact assessments on LLM adoption in various industries to understand societal implications.
5. Collaborate with regulatory bodies to establish guidelines for the responsible use of LLMs in applications.

**Discussion Angles:**
1. Explore the implications of LLMs on content creation, user interaction, and information dissemination.
2. Delve into the societal impacts of widespread LLM adoption on job markets, education, and communication.
3. Consider regulatory challenges and policy implications of integrating LLMs into existing systems.
4. Discuss collaborative opportunities for industry partnerships to advance LLM capabilities.
5. Analyze the long-term effects of LLMs on human-machine interaction and decision-making processes.

By addressing these strategic talking points, questions, and discussion angles, we can gain a comprehensive understanding of the impact of large language models on human life and drive meaningful discussions towards actionable insights and decisions.